# Operator seluler Megaline


# Daftar Isi <a id='back'></a>

* [Pendahuluan](#intro)

* [Tahap 1. Ikhtisar Data](#data_review)
  
* [Tahap 2. Analisis](#analysis)
    * [2.1 Membagi set train, set valid, dan set test](#als_train_valid_test)
    * [2.2 Pengujian Model](#als_uji_model)
    * [2.2.1 Decision Tree](#als_uji_dec_tree)
    * [2.2.2 Random Forest](#als_uji_rand_for)
    * [2.2.3 Logistic Regression](#als_uji_log_regr)
    * [2.3 Quality Check menggunakan set test](#als_quality_check)
    * [2.4 Sanity Check: Model dibandingkan dengan Peluang](#als_sanity_check)

* [Temuan](#end)

## Pendahuluan <a id='intro'></a>

Operator seluler Megaline merasa tidak puas karena banyak user mereka yang masih menggunakan paket lama. Perusahaan tersebut ingin mengembangkan sebuah model yang dapat menganalisis perilaku konsumen dan merekomendasikan salah satu dari kedua paket terbaru Megaline: Surf atau Ultimate.

Peneliti memiliki akses terhadap data perilaku para user yang sudah beralih ke paket terbaru (dari proyek kursus Analisis Data Statistik). Dalam tugas klasifikasi ini, Peneliti perlu mengembangkan sebuah model yang mampu memilih paket dengan tepat.


### Tujuan: 
Peneliti akan mengembangkan model yang memilih paket yang tepat untuk user berdasarkan data perilaku pelanggan yang telah beralih ke paket baru, Smart atau Ultra. Peneliti mencari model yang setidaknya 75% akurat. Mengingat hal ini adalah tugas klasifikasi, kita akan menguji classifier Decision Tree, classifier Random Forest, dan classifier Logistic Regression.

Melakukan pengujian klasifikasi dengan model:
* Decision Tree
* Random Forest
* Logistic Regression


### Tahapan
Data perilaku user dari operator seluler Megaline yang disimpan dalam *file*:
- /datasets/users_behavior.csv

Mengingat Peneliti telah menyelesaikan langkah pra-pemrosesan data, Peneliti bisa langsung menuju ke tahap pembuatan model. Data tersebut dapat langsung diuji pada model klasifikasi yang akan digunakan dengan membaginya menjadi set data:
* set train
* set valid
* set test

Proyek ini akan terdiri dari dua tahap:
 1. Ikhtisar Data
 2. Analisis

 
[Kembali ke Daftar Isi](#back)

## Tahap 1. Ikhtisar Data <a id='data_review'></a>

Melakukan import library yang diperlukan, membuka data terkait yang akan dievaluasi, kemudian menjelajahi data tersebut.

**Step 1.1**
Melakukan import libraries dan modules yang diperlukan

In [1]:
import pandas as pd #untuk pengolahan dataframes
from sklearn.tree import DecisionTreeClassifier #untuk pengolahan model Decision Tree
from sklearn.ensemble import RandomForestClassifier #untuk pengolahan model Random Forest
from sklearn.linear_model import LogisticRegression #untuk pengolahan  model Logistic Regression
from sklearn.model_selection import train_test_split #untuk kebutuhan pembagian dataset
from sklearn.metrics import accuracy_score #untuk menghitung accuracy dari model yang digunakan

**Step 1.2**
Melakukan load data yang diperlukan dari file *users_behavior.csv*

In [2]:
try:
    df = pd.read_csv('users_behavior.csv')
except:
    games = pd.read_csv('/datasets/users_behavior.csv')
#membaca the csv file and konversi ke dataframe df

df.head() #melihat 5 baris awal dari dataframe df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()#menampilkan informasi umum dataframe df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Feature yang ada adalah kolom calls, minutes, messages, dan mb_used, yang akan digunakan untuk predict target, yakni kolom is_ultra, berdasarkan model yang telah di-train

[Kembali ke Daftar Isi](#back)

## Tahap 2. Analisis <a name='analysis'></a>

## Membagi set train, set valid, dan set test <a name="als_train_valid_test"></a>

**Step 2.1**
Kita perlu membagi set train, set valid, dan set test. Untuk melakukan hal ini, kita menggunakan fungsi train_test_split(). Fungsi ini membagi kumpulan data menjadi 2. Karena kita memerlukan 3 kumpulan, kita perlu melakukannya dua kali. Persentase kumpulan data asli diset menjadi 60, 20, dan 20 untuk masing-masing set train, set valid, dan dan set test.

In [4]:
df_train, df2 = train_test_split(df, test_size=0.4, random_state=12345)
#membagi df menjadi df_train (60%) dan df2 (40%)
df_valid, df_test = train_test_split(df2, test_size=0.5, random_state=12345)
#membagi df2 menjadi df_valid(50% dari df2, yang berarti 20% dari df) dan df_test(50% dari df2, yang berarti 20% dari df)

**Step 2.2**
Melihat informasi umum dari setiap set split

In [5]:
df_train.info() #informasi set training

<class 'pandas.core.frame.DataFrame'>
Index: 1928 entries, 3027 to 482
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
 4   is_ultra  1928 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [6]:
df_valid.info() #informasi set validation

<class 'pandas.core.frame.DataFrame'>
Index: 643 entries, 1386 to 3197
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [7]:
df_test.info() #informasi set test

<class 'pandas.core.frame.DataFrame'>
Index: 643 entries, 160 to 2313
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


Kita telah membagi kumpulan data asli menjadi set training (60%), set validation (20%), dan set test (20%). Kita sekarang perlu menentukan feature dan target untuk setiap set. Untuk feature-nya, kita akan mengambil seluruh dataframe yang dihapus kolom target-nya, yakni kolom is_ultra.

**Step 2.3**
Membuat feature dan target untuk setiap set baik set training, set validation, maupun set test.

In [8]:
train_features = df_train.drop('is_ultra', axis=1)
#membentuk feature train dengan menghapus kolom target dari set train

train_target = df_train['is_ultra']
#membentuk target train yang merupakan kolom is_ultra pada set train

valid_features = df_valid.drop('is_ultra', axis=1)
#membentuk feature valid dengan menghapus kolom target dari set valid

valid_target = df_valid['is_ultra']
#membentuk target valid yang merupakan kolom is_ultra pada set valid

test_features = df_test.drop('is_ultra', axis=1)
#membentuk feature test dengan menghapus kolom target dari set test

test_target = df_test['is_ultra']
#membentuk target test yang merupakan kolom is_ultra pada set test

Kita telah mendefinisikan feature dan target untuk setiap set sehingga dapat dilanjutkan dengan  pengujian model

## Pengujian Model <a name="als_uji_model"></a>

Kita akan melakukan pengujian klasifikasi model Decision Tree, klasifikasi model Random Forest, dan model klasifikasi Logistic Regression dan melatihnya dengan set train (menggunakan metode fit()) dan mengujinya pada set valid dengan membandingkan prediksi menggunakan feature dari set valid (menggunakan metode predict()) ke target sebenarnya dari set valid. 
Kita akan menyesuaikan masing-masing hyperparameter untuk mendapatkan nilai accuracy yang lebih tinggi, yang terakhir menjadi metrik untuk memilih model terbaik yang digunakan.

### Decision Tree <a name="als_uji_dec_tree"></a>

Kita akan menggunakan fungsi DecisiontreeClassifier(). Hyperparameter yang digunakan adalah random_state dan max_depth. Hyperparameter random_state harus sama secara keseluruhan sehingga kita akan memberikannya nilai tetap (12345). Hyperparameter max_depth adalah hyperparameter yang akan kita gunakan lebih dalam. Kita akan mengulang sekumpulan nilai untuk max_depth (dalam hal ini 1 hingga 10) dan mendapatkan skor akurasinya untuk mencari mana yang terbaik.

**Step 2.4**
Membuat for loop untuk melakukan pengujian model Decision Tree

In [9]:
for i in range(1, 11): #loop melalui nilai i dari 1 hingga 10    
    dt_model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    #membuat model Decision Tree dengan nilai max_depth berupa i
    dt_model.fit(train_features, train_target)
    #train model dengan menggunakan feature dan target dari set train
    dt_valid_pred=dt_model.predict(valid_features)
    #mendapatkan prediction dari model dengen menggunakan feature dan target dari set valid
    print('Max depth', i, 'accuracy =', accuracy_score(valid_target, dt_valid_pred))
    #mencetak nilai accuracy dengan membandingkan prediction terhadap target dari set valid

Max depth 1 accuracy = 0.7542768273716952
Max depth 2 accuracy = 0.7822706065318819
Max depth 3 accuracy = 0.7853810264385692
Max depth 4 accuracy = 0.7791601866251944
Max depth 5 accuracy = 0.7791601866251944
Max depth 6 accuracy = 0.7838258164852255
Max depth 7 accuracy = 0.7822706065318819
Max depth 8 accuracy = 0.7791601866251944
Max depth 9 accuracy = 0.7822706065318819
Max depth 10 accuracy = 0.7744945567651633


Model Decision Tree terbaik adalah yang memiliki max_ depth 3 karena memiliki nilai akurasi tertinggi sebesar 79%

### Random Forest <a name="als_uji_rand_for"></a>

Kita menjalankan RandomForestClassifier(). Hyperparameter random_state kita harus tetap sama seperti sebelumnya. Hyperparameter yang akan kita mainkan adalah max_ depth dan n_estimators. Dalam hal ini pertama-tama kita akan membuat daftar kosong. Kemudian kita akan membuat loop nilai max_depth dan, di dalam loop tersebut dilakukan pengulangan penerapan nilai n_estimators. Kita akan menggunakan loop ini untuk membuat model dengan permutasi berbeda dari nilai max_ depth dan n_estimators yang akan kita simpan dalam daftar, di mana selanjutnya kita akan memilih model dengan nilai accuracy tertinggi.

**Step 2.5**
Membuat for loop untuk melakukan pengujian model Random Forest terbaik

In [10]:
rf = []
#membuat list kosong
for i in range(1, 11):
#loop melalui nilai i dari 1 hingga 10 untuk max_depth
    for j in range(10, 101, 10):
    #loop melalui nilai j dari 1 hingga 100 dengan step 10 untuk n_estimators
        rf_model = RandomForestClassifier(random_state=12345, max_depth=i, n_estimators=j)
        #membuat model random forest
        rf_model.fit(train_features, train_target)
        #trains model menggunakan feature dan target dari set train
        rf.append(rf_model)
        #menambahkan model ke dalam daftar
    
print(max(rf, key=lambda rf_model: accuracy_score(rf_model.predict(valid_features), valid_target)))
#mencetak model dengan nilai accuracy tertinggi berdasarkan prediction yang dihitung dengan menggunakan feature dari set valid dan target aktual dari set valid

RandomForestClassifier(max_depth=8, n_estimators=40, random_state=12345)


Hasil menunjukkan bahwa model klasifikasi Random Forest terbaik adalah model dengan max_ depth=8 dan n_estimators=40. Kita sebut sebagai best_rf. 

**Step 2.6**
Menghitung nilai akurasi untuk model klasifikasi Random Forest terbaik dengan menggunakan best_rf

In [11]:
best_rf=RandomForestClassifier(random_state=12345, max_depth=8, n_estimators=40)
best_rf.fit(train_features, train_target)
best_rf_pred=best_rf.predict(valid_features)
print(accuracy_score(valid_target, best_rf_pred))

0.8087091757387247


Klasifikasi Random Forest terbaik memiliki nilai accuracy sebesar 81%

### Logistic Regression <a name="als_uji_log_regr"></a>

Kita menggunakan fungsi LogisticRegression(). Hyperparameter random_state harus sama, namun hyperparameter max_ depth dan n_estimators tidak berlaku di sini. Yang kita perlukan hanyalah menetapkan hyperparameter solver. Untuk hal tersebut kita menggunakan 'liblinear'.

**Step 2.7**
Train model Logistic Regression dan menghitung nilai accuracy model tersebut

In [12]:
lr_model = LogisticRegression(random_state=12345, solver='liblinear')
lr_model.fit(train_features, train_target)
lr_valid_pred=lr_model.predict(valid_features)
print('Logistic Regression Accuracy =', accuracy_score(valid_target, lr_valid_pred))

Logistic Regression Accuracy = 0.7076205287713841


Nilai accuracy model Logistic Regression model adalah 76%

**Model manakah yang terbaik?**

Model terbaik yang kami dihasilkan adalah model Random Forest dengan max_ depth = 8 dan n_estimators = 40 dengan nilai accuracy 81%. Di posisi kedua adalah model Decision Tree dengan max_ depth = 3 dengan nilai accuracy 79%. Dan di posisi terakhir adalah model Regresi Logistik dengan nilai accuracy 76%.

## Quality Check menggunakan set test <a name="als_quality_check"></a>

Kita sekarang  menggunakan model terbaik pada set test. Sebelumnya, kita perlu melatih ulang model tersebut menggunakan gabungan set train dan set valid. Untuk menggabungkan set tersebut, kita dapat menggunakan fungsi pd.concat dengan menjadikan daftar set yang dipanggil sebagai argumen dan mengatur parameter axis=0 untuk menjadikannya sebagai tumpukan vertikal.

**Step 2.8**
Menggabungkan set train dan set valid

In [13]:
train_final = pd.concat([df_train, df_valid], axis=0)
#menumpuk secara vertikal set train dan set valid
train_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2571 entries, 3027 to 3197
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     2571 non-null   float64
 1   minutes   2571 non-null   float64
 2   messages  2571 non-null   float64
 3   mb_used   2571 non-null   float64
 4   is_ultra  2571 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 120.5 KB


**Step 2.9**
Menetapkan feature dan target untuk set gabungan tersebut

In [14]:
train_final_features = train_final.drop('is_ultra', axis=1)
train_final_target = train_final['is_ultra']

Sekarang kita dapat melatih (metode fit()) menggunakan feature dan target yang baru (gabungan train dan valid), yang dilanjutkan dengan membuat prediction (metode predict()) menggunakan feature dari set test, dan mendapatkan nilai akurasi. Model terbaik tetap disebut sebagai 'best_rf'

**Step 2.10**
Melatih model, menggunakan feature dan target gabungan, membuat prediction, dan menghitung nilai akurasi.

In [15]:
best_rf.fit(train_final_features, train_final_target)
best_rf_pred=best_rf.predict(test_features)
print(accuracy_score(best_rf_pred, test_target))

0.7993779160186625


Nilai accuracy yang didapatkan adalah 80% melebihi ambang batas (threshold) 75%.

## Sanity Check: Model dibandingkan dengan Peluang <a name="als_sanity_check"></a>

Untuk melakukan Sanity Check terhadap model kita, perlu dilakukan perbandingan dengan peluang. Kita dapat melakukannya dengan mendapatkan nilai akurasi jika prediction kita mengasumsikan bahwa kita memberikan satu nilai untuk seluruh target. Untuk hal tersebut, kita lihat berapa banyak user Smart dan Ultra yang kita miliki pada set test.

**Step 2.11**
Melihat data jumlah pengguna masing-masing paket

In [16]:
smart_target=(test_target == 0) #nilai True untuk pengguna paket Smart
ultra_target=(test_target == 1) #nilai True untuk pengguna paket Ultra

print('Jumlah pengguna paket Smart:', smart_target.sum())
print('Jumlah pengguna paket Ultra:', ultra_target.sum())

Jumlah pengguna paket Smart: 440
Jumlah pengguna paket Ultra: 203


Kita memiliki lebih banyak user paket Smart, untuk selanjutnya kita akan menggunakan mereka sebagai contoh, yakni dengan mengasumsikan kita memiliki random classifier dengan prediction hanya 0 (yaitu paket Smart) selama set test. Kemudian dicari nilai accuracy-nya.

**Step 2.12**
Menghitung peluang paket Smart

In [17]:
peluang_smart=smart_target.sum()/len(test_target)
print('Nilai accuracy dari random classifier:', peluang_smart)

Nilai accuracy dari random classifier: 0.6842923794712286


Random classifier memiliki skor akurasi 68,4%, kurang dari 80% yang diperoleh classifier Random Forest sebelumnya. Jadi model kita lolos Sanity Check.

[Kembali ke Daftar Isi](#back)

## Temuan <a name="end"></a>

Kita telah membagi data menjadi set train, set valid, dan set test. Kita telah menguji model dan menemukan bahwa RandomForestClassifier adalah yang terbaik, mendapatkan nilai accuracy 81% pada set valid dan 79% pada set test dan lulus pemeriksaan sanity check.

[Kembali ke Daftar Isi](#back)